In [321]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import pickle
import csv
import implicit
import itertools
import copy

In [322]:
df = pd.read_csv("../data/readers.csv")
df = df.rename(columns={"id":"uid", "art_id":"mid"})
df.head()

,uid,mid
0,1,ld.154103
1,1,ld.142559
2,1,1.18331199
3,1,ld.144819
4,1,ld.1293110


In [323]:
n_users = df.uid.unique().shape[0]
n_items = df.mid.unique().shape[0]

print('Number of users: {}'.format(n_users))
print('Number of models: {}'.format(n_items))
print('Sparsity: {:4.3f}%'.format(float(df.shape[0]) / float(n_users*n_items) * 100))

Number of users: 1000
Number of models: 11260
Sparsity: 0.247%


In [324]:
def threshold_likes(df, uid_min, mid_min):
    n_users = df.uid.unique().shape[0]
    n_items = df.mid.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Starting likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        mid_counts = df.groupby('uid').mid.count()
        df = df[~df.uid.isin(mid_counts[mid_counts < mid_min].index.tolist())]
        uid_counts = df.groupby('mid').uid.count()
        df = df[~df.mid.isin(uid_counts[uid_counts < uid_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby('uid').mid.count().min() >= mid_min)
    assert(df.groupby('mid').uid.count().min() >= uid_min)
    
    n_users = df.uid.unique().shape[0]
    n_items = df.mid.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    return df

In [325]:
df_lim = threshold_likes(df, 5, 5)

Starting likes info
Number of users: 1000
Number of models: 11260
Sparsity: 0.247%
Ending likes info
Number of users: 795
Number of models: 1315
Sparsity: 1.020%


In [326]:
# Create mappings
mid_to_idx = {}
idx_to_mid = {}
for (idx, mid) in enumerate(df_lim["mid"].unique().tolist()):
    mid_to_idx[mid] = idx
    idx_to_mid[idx] = mid
    
uid_to_idx = {}
idx_to_uid = {}
for (idx, uid) in enumerate(df_lim["uid"].unique().tolist()):
    uid_to_idx[uid] = idx
    idx_to_uid[idx] = uid

In [327]:
def map_ids(row, mapper):
    return mapper[row]

In [328]:
I = df_lim["uid"].apply(map_ids, args=[uid_to_idx]).to_numpy()
J = df_lim["mid"].apply(map_ids, args=[mid_to_idx]).to_numpy()
V = np.ones(I.shape[0])
likes = sparse.coo_matrix((V, (I, J)), dtype=np.float64)
likes = likes.tocsr()

In [329]:
def train_test_split(ratings, split_count, fraction=None):
    """
    Split recommendation data into train and test sets
    
    Params
    ------
    ratings : scipy.sparse matrix
        Interactions between users and items.
    split_count : int
        Number of user-item-interactions per user to move
        from training to test set.
    fractions : float
        Fraction of users to split off some of their
        interactions into test set. If None, then all 
        users are considered.
    """
    # Note: likely not the fastest way to do things below.
    train = ratings.copy().tocoo()
    test = sparse.lil_matrix(train.shape)
    
    if fraction:
        try:
            user_index = np.random.choice(
                np.where(np.bincount(train.row) >= split_count * 2)[0], 
                replace=False,
                size=np.int32(np.floor(fraction * train.shape[0]))
            ).tolist()
        except:
            print(('Not enough users with > {} '
                  'interactions for fraction of {}')\
                  .format(2*k, fraction))
            raise
    else:
        user_index = range(train.shape[0])
        
    train = train.tolil()

    for user in user_index:
        test_ratings = np.random.choice(ratings.getrow(user).indices, 
                                        size=split_count, 
                                        replace=False)
        train[user, test_ratings] = 0.
        # These are just 1.0 right now
        test[user, test_ratings] = ratings[user, test_ratings]
   
    
    # Test and training are truly disjoint
    assert(train.multiply(test).nnz == 0)
    return train.tocsr(), test.tocsr(), user_index

In [330]:
train, test, user_index = train_test_split(likes, 5, fraction=0.2)

In [331]:
train.shape

(795, 1315)

In [332]:
test.shape

(795, 1315)

In [333]:
def precision_at_k(model, ratings, k=5, user_index=None):
    if not user_index:
        user_index = range(ratings.shape[0])

    ratings = ratings.tocsr()
    precisions = []
    # Note: line below may become infeasible for large datasets.
    for user in user_index:
        # In case of large dataset, compute predictions row-by-row like below
        # predictions = np.array([model.predict(row, i) for i in xrange(ratings.shape[1])])
        #top_k = np.argsort(-predictions[user, :])[:k]
        top_k = model(user, N=k)
        #print(top_k)
        #top_k = [idx for idx, confidence in top_k]
        labels = ratings.getrow(user).indices
        #print(len(set(top_k) & set(labels)))
        precision = float(len(set(top_k) & set(labels))) / float(k)
        precisions.append(precision)
    return np.mean(precisions)   

In [406]:
def recall_at_k(model, ratings, k=5, user_index=None):
    if not user_index:
        user_index = range(ratings.shape[0])

    recalls = []
    for user in user_index:
        top_k = model(user, N=k)
        #print(top_k)
        #top_k = [idx for idx, confidence in top_k]
        labels = test.getrow(user).indices
        #print(labels)
        #print(len(set(top_k) & set(labels)))
        recall = float(len(set(top_k) & set(labels))) / float(len(set(labels)))
        recalls.append(recall)
    
    return np.mean(recalls)


In [407]:
## CF Model
from scipy.sparse.linalg import svds

n_latent_factors = 100

U, sigma, Vt = svds(train, k=n_latent_factors)
sigma = np.diag(sigma)

reader_predictions = np.dot(np.dot(U, sigma), Vt)
reader_predictions_norm = (reader_predictions - reader_predictions.min()) / (
        reader_predictions.max() - reader_predictions.min()
)



In [408]:
indices = reader_predictions_norm[1].argsort()[-10:][::-1]

In [409]:
def recommend(user_id, N):
    recoms = reader_predictions_norm[user_id].argsort()[-N:][::-1]
    return recoms
    

In [410]:
recommend(1, 100)

array([  15,    9,   12,   16,   10,    8,    7,   13,   11, 1066,   14,
         18,  236,  600,  230,  766,  735,  604,  601,   17,  936, 1280,
        857,  972,  885,  605, 1215,  768, 1074,  351,  779,   94, 1180,
        656,  461,  720,  239, 1289,  879,  273,  340,  485,  983, 1059,
        208, 1130,  494,  635,  584, 1093, 1003,  680,  247,  943,  938,
       1244, 1112,  568,  123,  971,   75, 1083,  542,  772, 1085,  823,
        102,  655,  332,  969,  382, 1173, 1076,  719,  434, 1155,   67,
       1024,  117,  218,  321,  593,  982,  617, 1021,   28,  354,  773,
        871,   38,  560,   79,  267,  572, 1163,  516,  863,  853, 1051,
        536], dtype=int64)

In [411]:
print(precision_at_k(recommend, test,k=10, user_index=user_index))
print(recall_at_k(recommend, test,k=5, user_index=user_index))
print(recall_at_k(recommend, test,k=10, user_index=user_index))


0.019496855345911952
0.017610062893081764
0.038993710691823905


In [405]:
for index in indices:
    print(idx_to_mid[index])

ld.150497
ld.150557
ld.147049
ld.147794
ld.141732
ld.1293241
ld.152926
ld.138548
ld.141745
ld.142444


In [52]:
train.getrow(3).indices

array([32, 33, 34, 35, 36, 37, 38, 39])